<a href="https://colab.research.google.com/github/KwonDuHyeon/hanghae99/blob/main/4%EC%A3%BC%EC%B0%A8%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C(%EA%B6%8C%EB%91%90%ED%98%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4주차 심화과제 수능문제

In [ ]:
#!pip install transformers datasets evaluate accelerate scikit-learn

## Import 부분

In [ ]:
import random
import evaluate
import numpy as np
import pandas as pd
import os
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/harry/anaconda3/envs/hanghae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/harry/anaconda3/envs/hanghae/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/harry/anaconda3/envs/hanghae/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_glo

## [MY CODE] 수능국어 데이터 load


In [ ]:
Korean_data_path = "/mnt/d/2023_11_KICE.json"
with open(Korean_data_path, 'r', encoding='utf-8') as file:
    Korean_data = json.load(file)

## [MY CODE] 수능 국어 데이터 확인

In [ ]:
# type 확인
print(type(Korean_data))

<class 'list'>


In [ ]:
sample_Korean = Korean_data[0]
print(json.dumps(sample_Korean, indent=2, ensure_ascii=False))

{
  "id": "2023_11_KICE_1-3",
  "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도

## [MY CODE] GPT API 호출

In [ ]:
ls -l /mnt/d/hanghae99/apikey/.env

-rwxrwxrwx 1 harry harry 180 Jan  9 13:20 /mnt/d/hanghae99/apikey/.env*


In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv('/mnt/d/hanghae99/apikey/.env')  # .env 파일을 로드



# 환경 변수 확인
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Model.list()


<OpenAIObject list at 0x7fde93999770> JSON: {
  "object": "list",
  "data": [
    {
      "id": "gpt-4o-audio-preview-2024-10-01",
      "object": "model",
      "created": 1727389042,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-mini-audio-preview",
      "object": "model",
      "created": 1734387424,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-realtime-preview",
      "object": "model",
      "created": 1727659998,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-realtime-preview-2024-10-01",
      "object": "model",
      "created": 1727131766,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-mini-audio-preview-2024-12-17",
      "object": "model",
      "created": 1734115920,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-mini-realtime-preview",
      "object": "model",
      "created": 1734387380,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
 

In [ ]:
import time
def safe_api_call(prompt, max_tokens=5, retries=3, model="gpt-4o", delay=10):
    """
    OpenAI API 호출을 안전하게 처리하는 함수.
    RateLimitError 및 기타 오류 발생 시 재시도 로직 포함.
    """
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,


            )
            return response  # 성공적으로 응답 반환
        except openai.error.RateLimitError as e:
            print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt + 1}/{retries})")
            time.sleep(delay)  # 대기 후 재시도
        except Exception as e:
            print(f"Unexpected error: {e}. Retrying... (Attempt {attempt + 1}/{retries})")
            time.sleep(delay)  # 대기 후 재시도

    # 최대 재시도 횟수 초과 시 예외 발생
    raise Exception("Failed to get a response from OpenAI API after multiple retries.")


## [MY CODE] 결과 비교 함수 정의

In [ ]:
# 2. 점수 계산 함수
def calculate_score(predictions, problems):
    total_score = 0
    results = []  # 문제별 결과 저장
    for idx, (pred, problem) in enumerate(zip(predictions, problems)):
        correct_answer = problem["answer"]
        is_correct = pred == correct_answer
        total_score += problem["score"] if is_correct else 0

        # 문제별 결과 저장
        results.append({
            "문제 번호": idx + 1,
            "GPT-4 예측": pred,
            "정답": correct_answer,
            "예측 결과": "정답" if is_correct else "오답",
            "배점": problem["score"] if is_correct else 0
        })
    return total_score, results


## [MY CODE] prediction 함수 정의

In [ ]:
import re

    # 4. prediction 함수 정의
def prediction(problem):
    """
    Safe API Call을 사용하여 문제에 대한 예측 결과 반환.
    """
    plus =  problem.get('question_plus','')

    prompt = f"""다음은 수능 국어 문제입니다. 가장 적절한 답을 선택하세요.
     본문:
    {problem['paragraph']}

    질문:
    {problem['question']}
    {plus}
    선택지:
    {', '.join(f'({i + 1}) {choice}' for i, choice in enumerate(problem['choices']))}

    "1, 2, 3, 4, 5 중 하나의 숫자로만 답하세요. 다른 정보는 포함하지 마세요."
    """

    # Safe API 호출로 응답 받기
    response = safe_api_call(prompt, max_tokens=5, retries=5, model="gpt-4o", delay=10)


    # GPT-4의 응답에서 정답 추출
    gpt_answer = response['choices'][0]['message']['content'].strip()


    # 정규 표현식으로 숫자 추출 (1-5에 해당하는 숫자만)
    match = re.search(r'\b[1-5]\b', gpt_answer)
    if match:
        return int(match.group())  # 숫자를 반환
    else:
        # 오류 발생 시 1번으로 가정 (예외 처리)
        return 1

## [MY CODE] 실행

In [ ]:

# 문제 데이터 처리
problems = []
for section in Korean_data:
    for problem in section["problems"]:
        # 각 문제에 상위 paragraph 추가
        problem["paragraph"] = section["paragraph"]
        problems.append(problem)

# 모든 문제에 대해 GPT-4 예측 실행 (순차적으로 처리)
predictions = []
for idx, problem in enumerate(problems):
    try:
        print(f"Processing problem {idx + 1}/{len(problems)}...")
        pred = prediction(problem)  # 문제 예측
        predictions.append(pred)
        time.sleep(2)  # 요청 간 1초 대기
    except Exception as e:
        print(f"Error processing problem {idx + 1}: {e}")
        predictions.append(1)  # 실패 시 기본값

# 점수 계산 및 결과 출력
final_score, results = calculate_score(predictions, problems)

# 문제별 결과 출력
print("\n문제별 예측 결과:")
for result in results:
    print(f"문제 {result['문제 번호']}: GPT-4 예측 = {result['GPT-4 예측']}, "
          f"정답 = {result['정답']}, 결과 = {result['예측 결과']}, 배점 = {result['배점']}")

# 최종 점수 출력
print(f"\nGPT-4 최종 점수: {final_score}")

Processing problem 1/45...
Processing problem 2/45...
Processing problem 3/45...
Processing problem 4/45...
Processing problem 5/45...
Processing problem 6/45...
Processing problem 7/45...
Processing problem 8/45...
Processing problem 9/45...
Processing problem 10/45...
Processing problem 11/45...
Processing problem 12/45...
Processing problem 13/45...
Processing problem 14/45...
Processing problem 15/45...
Processing problem 16/45...
Processing problem 17/45...
Processing problem 18/45...
Processing problem 19/45...
Processing problem 20/45...
Processing problem 21/45...
Processing problem 22/45...
Processing problem 23/45...
Processing problem 24/45...
Processing problem 25/45...
Processing problem 26/45...
Processing problem 27/45...
Processing problem 28/45...
Processing problem 29/45...
Processing problem 30/45...
Processing problem 31/45...
Processing problem 32/45...
Processing problem 33/45...
Processing problem 34/45...
Processing problem 35/45...
Processing problem 36/45...
P

## [LOG] question plus 추가후 점수 96점

## [MY CODE] 프롬프트2 구체적인 지시를 추가

In [ ]:
import time
from nltk.tokenize import sent_tokenize
def safe_api_call(prompt, max_tokens=5, retries=3, model="gpt-4o", delay=60):
    """
    OpenAI API 호출을 안전하게 처리하는 함수.
    RateLimitError 및 기타 오류 발생 시 재시도 로직 포함.
    """
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.2  # 약간의 변동성 허
            )
            return response  # 성공적으로 응답 반환
        except openai.error.RateLimitError as e:
            print(f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt + 1}/{retries})")
            time.sleep(delay)  # 대기 후 재시도
        except Exception as e:
            print(f"Unexpected error: {e}. Retrying... (Attempt {attempt + 1}/{retries})")
            time.sleep(delay)  # 대기 후 재시도

    # 최대 재시도 횟수 초과 시 예외 발생
    raise Exception("Failed to get a response from OpenAI API after multiple retries.")


# 4. prediction 함수 정의
def prediction(problem):
    plus =  problem.get('question_plus','')
     # 지문 요약
    prompt = f"""다음은 대한민국 수능 국어 문제입니다. 본문과 문제를 분석하여 가장 적절한 답을 선택하세요.

    ### 본문 ###
    {problem['paragraph']}

    ### 질문 ###
    {problem['question']}
    {plus}
    ### 선택지 ###
    {', '.join(f'({i + 1}) {choice}' for i, choice in enumerate(problem['choices']))}


     ### 작업 지침 ###
    1. 본문과 질문을 논리적으로 분석합니다.
    2. 각 선택지와 본문을 비교하여, 가장 근거가 명확한 선택지를 고르세요.
    3. 불확실한 경우, 본문에서 가장 관련성이 높은 단서를 포함한 선택지를 선택하세요.


    ### 답변 형식 ###
    - 정답은 반드시 숫자로만 작성하세요. 예: 1, 2, 3, 4, 5
    - 다른 정보는 포함하지 마세요.
    """

    # Safe API 호출로 응답 받기
    response = safe_api_call(prompt, max_tokens=5, retries=5, model="gpt-4o", delay=10)

    # GPT-4의 응답에서 정답 추출
    gpt_answer = response['choices'][0]['message']['content'].strip()

     # 숫자가 아닌 경우 또는 범위를 벗어난 경우 예외 발생
    if not gpt_answer.isdigit() or int(gpt_answer) not in range(1, 6):
        raise ValueError(f"Unexpected response: {gpt_answer}")

    # 정답 반환
    return int(gpt_answer)


# 문제 데이터 처리
problems = []
for section in Korean_data:
    for problem in section["problems"]:
        # 각 문제에 상위 paragraph 추가
        problem["paragraph"] = section["paragraph"]
        problems.append(problem)

# 모든 문제에 대해 GPT-4 예측 실행 (순차적으로 처리)
predictions = []
for idx, problem in enumerate(problems):
    try:
        print(f"Processing problem {idx + 1}/{len(problems)}...")
        pred = prediction(problem)  # 문제 예측
        predictions.append(pred)
        time.sleep(2)  # 요청 간 2초 대기
    except Exception as e:
        print(f"Error processing problem {idx + 1}: {e}")
        predictions.append(1)  # 실패 시 기본값

# 점수 계산 및 결과 출력
final_score, results = calculate_score(predictions, problems)

# 문제별 결과 출력
print("\n문제별 예측 결과:")
for result in results:
    print(f"문제 {result['문제 번호']}: GPT-4 예측 = {result['GPT-4 예측']}, "
          f"정답 = {result['정답']}, 결과 = {result['예측 결과']}, 배점 = {result['배점']}")

# 최종 점수 출력
print(f"\nGPT-4 최종 점수: {final_score}")


Processing problem 1/45...
Processing problem 2/45...
Processing problem 3/45...
Processing problem 4/45...
Processing problem 5/45...
Processing problem 6/45...
Processing problem 7/45...
Processing problem 8/45...
Processing problem 9/45...
Processing problem 10/45...
Processing problem 11/45...
Processing problem 12/45...
Processing problem 13/45...
Processing problem 14/45...
Processing problem 15/45...
Processing problem 16/45...
Processing problem 17/45...
Processing problem 18/45...
Processing problem 19/45...
Processing problem 20/45...
Processing problem 21/45...
Processing problem 22/45...
Processing problem 23/45...
Processing problem 24/45...
Processing problem 25/45...
Processing problem 26/45...
Processing problem 27/45...
Processing problem 28/45...
Processing problem 29/45...
Processing problem 30/45...
Processing problem 31/45...
Processing problem 32/45...
Processing problem 33/45...
Processing problem 34/45...
Processing problem 35/45...
Processing problem 36/45...
P

## [LOG] 오히려 성능이 더 떨어졌다 작업 지침 내용수정 후 다시 시도해볼만하다.
추후 개인적으로 해서 성능 개선 연습을 진행하겠습니다.